# Execute all cells

In [ ]:
import os
os.getcwd()
os.chdir('/work')
print(os.getcwd())
current_folder = globals()['_dh'][0]
import pathlib
import sys
del sys.argv[1]
print(sys.path)

from experiments.transformations.analysis.activation_distance.activation_distance_utils import *

In [ ]:
import pickle
import sys
from sty import fg, rs, bg, ef
def save_act_dist_shapenet(pt_transform, distance_transform, seed=0, num_output_net=None):
    vp = 50 if 'v' in pt_transform else 1
    if pt_transform == 'vanilla':
        print(fg.red+ "Loading vanilla net"+rs.fg)
        exp.pretraining = 'vanilla'
        exp.network_name = network_name
        net = exp.get_net(1000)
        net_name = 'vanilla'
        folder_output = 'others'
        
    elif pt_transform == 'ImageNet':
        exp.network_name = 'vgg11bn'
        exp.pretraining = './models/ptImageNet/vgg11bn.pt'
        net = exp.get_net(1000)
        net_name = 'ImageNet'
        folder_output = 'others'
    else:
        if dataset == 'ShapeNet':
            net_name = f'FULLTRAIN_T{pt_transform}_objs{objs}_vp{vp}_mat0_trset1_S{seed}'
        elif dataset == 'cifar10':
            net_name = f'FULLTRAIN_T{pt_transform}_imgs{objs}_{network_name}_S{seed}'
        exp.network_name = network_name
        exp.pretraining = f'./models/transformations/{dataset}/{network_name}/{folder}/{net_name}.pt'
        folder_output = folder
        net = exp.get_net(dataset_no_transf.num_classes if num_output_net is None else num_output_net)
    if network_name == 'samediffnet':
        net = net.backbone

    net.eval()
    net.cuda()
    if dataset == 'cifar10':
        log_dataset = 'cifar10set2'
    elif dataset == 'ShapeNet':
        log_dataset = 'shapeNetSet2'
    else:
        assert False, 'Type Dataset Not Recognized'

    distAct = get_distance_fun(distance_transform)

    dist = distAct(net, dataset_no_transf, use_cuda=exp.use_cuda, distance=distance, compare_with=CompareWith.SAME_OBJECT)
    distance_net_same, distance_img, x_values = dist.calculate_distance_dataloader()

    dist = distAct(net, dataset_no_transf, use_cuda=exp.use_cuda, distance=distance, compare_with=CompareWith.ANY_OBJECT)
    distance_net_any_objs, distance_img_any_objs, x_values_any_objs = dist.calculate_distance_dataloader()

    dist = distAct(net, dataset_no_transf, use_cuda=exp.use_cuda, distance=distance, compare_with=CompareWith.SAME_CLASS_DIFF_OBJ)
    distance_net_same_class_diff_obj, distance_img_diff, x_values_diff = dist.calculate_distance_dataloader()

    output_path = f'./results/transformations/activation_distance/{dataset}/{network_name}/{folder_output}/{distance}_{distance_transform}_{log_dataset}_{net_name}.pickle'
    print(fg.red + f"Saving result in {output_path}" + rs.fg)
    pathlib.Path(os.path.dirname(output_path)).mkdir(parents=True, exist_ok=True)

    pickle.dump({'distance_net_same_obj': distance_net_same,
                 'distance_net_same_class_diff_obj': distance_net_same_class_diff_obj,
                 'distance_net_any_obj': distance_net_any_objs,
                 'distance_img': distance_img, 'x_values': x_values}, open(output_path,'wb'))




# Run the following cell to use measure invariance on the ShapeNet dataset

In [3]:
objs=50 # this doesn't need to match your network. Leave it at 50
type_dataset = 'different_classes'
exp, dataset_no_transf = get_experiment_shapeNet(type_dataset)

Using cuda - you are probably on the server
Running experiment in SCRIPT mode. Command line arguments will have the precedence over keywords arguments.
Device Name: [GeForce RTX 3090] - Selected device num: 3

**LIST_TAGS**:
['ptvanilla', 'bkbnvanilla']
***PARAMS***
	add_cat_module : False
	additional_tags : None
	backbone_name : None
	batch_size : 32
	classes_set : id_set2
	device_name : GeForce RTX 3090
	diff_classes_set : None
	do_train : True
	force_cuda : None
	freeze_all_but_last : False
	freeze_backbone : False
	learning_rate : None
	max_epochs : -1
	max_iterations : -1
	max_objs_per_class_test : None
	max_objs_per_class_train : 3
	model_output_filename : None
	network_name : vgg11bn
	num_iterations_testing : -1
	num_runs : 1
	num_viewpoints_test : None
	num_viewpoints_train : None
	output_filename : None
	patience_stagnation : -1
	pretraining : vanilla
	pretraining_backbone : vanilla
	project_name : TestProject
	scramble_fc : 0
	seed : 1
	stop_when_train_acc_is : -1
	transforma

# Run the following cell to compute the cosine similarities
Notice that you can use either 'cossim' or 'euclidean' distance. In the paper we use 'cossim' distance.
Change `network_names`, `seed` and `all_objs_values` to match the network you have trained.

In [ ]:
dataset = 'ShapeNet'
folder = 'fulltrain'
distance = 'cossim' # of 'euclidean'

# Suggested Parameters
seeds = [1]
objs_list = [50]
network_names = ['samediffnet']

# Paper Parameters
# seeds = [1, 2, 3]
# all_objs_values = [5, 50, 100, 250, 500]
# network_names = ['resnet18', 'resnet50', 'vgg11bn', 'vgg19bn', 'alexnet', 'googlenet', 'densenet201', 'samediffnet']
for network_name in network_names:
    for objs in objs_list:
        print(f"DOING OBJS {objs}")

        [save_act_dist_shapenet('v', 'VP', s) for s in seeds]
        [save_act_dist_shapenet('r', 'Rotate', s) for s in seeds]
        [save_act_dist_shapenet('s', 'Scale', s) for s in seeds]
        [save_act_dist_shapenet('t', 'Translate', s) for s in seeds]
        [save_act_dist_shapenet('b', 'Brightness', s) for s in seeds]
        [save_act_dist_shapenet('c', 'Contrast', s) for s in seeds]

        typeNet = ''
        [save_act_dist_shapenet(typeNet, 'VP', s) for s in seeds]
        [save_act_dist_shapenet(typeNet, 'Rotate', s) for s in seeds]
        [save_act_dist_shapenet(typeNet, 'Scale', s) for s in seeds]
        [save_act_dist_shapenet(typeNet, 'Translate', s) for s in seeds]
        [save_act_dist_shapenet(typeNet, 'Brightness', s) for s in seeds]
        [save_act_dist_shapenet(typeNet, 'Contrast', s) for s in seeds]

DOING OBJS 250
Loading.. full model from ./models/transformations/ShapeNet/resnet18/fulltrain/FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S1.pt... Done.
Saving result in ./results/transformations/activation_distance/ShapeNet/resnet18/fulltrain/euclidean_VP_shapeNetSet2_FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S1.pickle
Loading.. full model from ./models/transformations/ShapeNet/resnet18/fulltrain/FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S2.pt... Done.
Saving result in ./results/transformations/activation_distance/ShapeNet/resnet18/fulltrain/euclidean_VP_shapeNetSet2_FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S2.pickle
Loading.. full model from ./models/transformations/ShapeNet/resnet18/fulltrain/FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S3.pt... Done.
Saving result in ./results/transformations/activation_distance/ShapeNet/resnet18/fulltrain/euclidean_VP_shapeNetSet2_FULLTRAIN_Tv_objs250_vp50_mat0_trset1_S3.pickle
Loading.. full model from ./models/transformations/ShapeNet/resnet18/fulltrain/FULLTRAIN_Tr_ob

# Santy Check:
Run the following cell to calculate the translation invarinace for a Vanilla VGG11bn network, and a Vgg11bn network pretrained on ImageNet

In [ ]:
objs = None
dataset = 'ShapeNet'
network_name = 'vgg11bn'
folder = 'fulltrain'
distance = 'cossim'
#distance = 'euclidean'

# Some Sanity Check
save_act_dist_shapenet('s', 'Rotate')
save_act_dist_shapenet('t', 'Rotate')

save_act_dist_shapenet('r', 'Scale')
save_act_dist_shapenet('t', 'Scale')

save_act_dist_shapenet('r', 'Translate')
save_act_dist_shapenet('s', 'Translate')


typeNet = 'vanilla'
save_act_dist_shapenet(typeNet, 'VP')
save_act_dist_shapenet(typeNet, 'Rotate')
save_act_dist_shapenet(typeNet, 'Scale')
save_act_dist_shapenet(typeNet, 'Translate')
save_act_dist_shapenet(typeNet, 'Brightness')
save_act_dist_shapenet(typeNet, 'Contrast')

typeNet = 'ImageNet'
save_act_dist_shapenet(typeNet, 'VP')
save_act_dist_shapenet(typeNet, 'Rotate')
save_act_dist_shapenet(typeNet, 'Scale')
save_act_dist_shapenet(typeNet, 'Translate')
save_act_dist_shapenet(typeNet, 'Brightness')
save_act_dist_shapenet(typeNet, 'Contrast')
save_act_dist_shapenet(typeNet, 'Translate')
